# Data quality check

In [25]:
# Imports
import json
import os, sys
import csv 
from pathlib import Path
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import dataframe_image as dfi
a4_dims = (11.7, 8.27)
palette = sns.color_palette('colorblind', 10)
palette = sns.color_palette('Paired', 10)
from dateutil import relativedelta
from datetime import datetime

In [26]:
def json_to_dict(path):
    """
    Helper function
    :param path: Path to json file
    :return: dictionary with json content
    """
    with open(path) as json_file:
        data = json.load(json_file)
    return data

In [27]:
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

path_csv = os.path.join(parent_dir,"outputs", "data", "small_sample.csv")
raw_csv = open(path_csv, encoding="utf-8-sig")
reader = csv.reader(raw_csv, delimiter=";")
content_list = list(reader)
repo_sample_df = pd.DataFrame(content_list[1:],columns=content_list[0])
path_metrics = os.path.join(parent_dir,"outputs", "results", "csv_metrics.json")
path_data = os.path.join(parent_dir,"outputs", "data")
metrics_dict = json_to_dict(path=path_metrics)

## Repository

In [28]:
columns = ["language", "feature", "time_parameter", "min_date", "max_date", "time_diff_days", "target_period"]
check_results = pd.DataFrame(columns = columns)
print(check_results)

Empty DataFrame
Columns: [language, feature, time_parameter, min_date, max_date, time_diff_days]
Index: []


In [29]:
filter_date = "2022-12-31"

for file in os.listdir(path_data):
    if "repository" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        language_set = set()
        archived_set = set()
        for repo, data in json_data.items():
            updated_at = data.get("updated_at")
            language = data.get("language")
            language_set.add(language)
            archived = data.get("archived")
            archived_set.add(archived)
            upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
            updated_range.append(upd_date)
        min_date = min(updated_range)
        max_date = max(updated_range)
        print(language_set)
        print(archived_set)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "repository",
                   "time_parameter": "updated_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": ">2022-12-31"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])


# path_repository = os.path.join(parent_dir,"outputs", "data", "csv_repository.json")

cpp_repository.json
1000
{'C++'}
{False}
2023-08-02
2023-08-23
-------------------
csv_repository.json
86
{'PHP', 'Perl', 'TypeScript', 'Python', 'JavaScript', 'C++', 'Ruby', 'C', 'C#', 'Java'}
{False}
2023-07-21
2023-08-25
-------------------
JavaScript_repository.json
1000
{'JavaScript'}
{False}
2023-08-09
2023-08-23
-------------------
java_repository.json
1000
{'Java'}
{False}
2023-08-03
2023-08-23
-------------------
php_repository.json
1000
{'PHP'}
{False}
2023-06-15
2023-08-23
-------------------


python_repository.json
1000
{'Python'}
{False}
2023-08-15
2023-08-23
-------------------


In [30]:
check_results

,language,feature,time_parameter,min_date,max_date,time_diff_days
0,cpp_repository.json,repository,updated_at,2023-08-02,2023-08-23,21
1,csv_repository.json,repository,updated_at,2023-07-21,2023-08-25,35
2,JavaScript_repository.json,repository,updated_at,2023-08-09,2023-08-23,14
3,java_repository.json,repository,updated_at,2023-08-03,2023-08-23,20
4,php_repository.json,repository,updated_at,2023-06-15,2023-08-23,69
5,python_repository.json,repository,updated_at,2023-08-15,2023-08-23,8


## Advisories

In [31]:
for file in os.listdir(path_data):
    if "advisories" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))

cpp_advisories.json
1000
csv_advisories.json
86
JavaScript_advisories.json
1000
java_advisories.json
1000
php_advisories.json
1000
python_advisories.json
1000


## Commits

In [32]:
for file in os.listdir(path_data):
    if "commits" in file and "single" not in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        language_set = set()
        archived_set = set()
        for repo, data in json_data.items():
            nr_commits = len(data)
            for commit in data:
                commit_elem = commit.get("commit")
                updated_at = commit_elem.get("committer").get("date")
                upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
                updated_range.append(upd_date)

        min_date = min(updated_range)
        max_date = max(updated_range)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "commits",
                   "time_parameter": "updated_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-12 months"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])

cpp_commits.json
1000
2022-08-23
2023-08-24
-------------------
csv_commits.json
86
2022-08-25
2023-08-25
-------------------
JavaScript_commits.json
1000
2022-08-23
2023-08-24
-------------------
java_commits.json
1000
2022-08-23
2023-08-24
-------------------
php_commits.json
1000
2022-08-23
2023-10-09
-------------------
python_commits.json
1000
2022-08-23
2023-08-26
-------------------


In [33]:
file_path = os.path.join(path_data, "python_repository.json")
json_data = json_to_dict(file_path)
file_commits = os.path.join(path_data, "python_commits.json")
json_commits = json_to_dict(file_commits)
python_repos = list(json_data.keys())
python_commit = list(json_commits.keys())
list(set(python_repos) - set(python_commit))

[]

36818673
https://api.github.com/repositories/36818673/commits?since=2023-07-23T00:00:00Z&per_page=100&page=1

610912437
https://api.github.com/repositories/610912437/commits?since=2023-07-23T00:00:00Z&per_page=100&page=1

1864194
https://api.github.com/repositories/1864194/commits?since=2023-07-23T00:00:00Z&per_page=100&page=1
https://api.github.com/repositories/1864194/commits?since=2023-07-23T00%3A00%3A00Z&per_page=100&page=2
https://api.github.com/repositories/1864194/commits?since=2023-07-23T00%3A00%3A00Z&per_page=100&page=3
https://api.github.com/repositories/1864194/commits?since=2023-07-23T00%3A00%3A00Z&per_page=100&page=4

In [34]:
file_path = os.path.join(path_data, "python_repository.json")
json_data = json_to_dict(file_path)
file_commits = os.path.join(path_data, "python_commits.json")
json_commits = json_to_dict(file_commits)
python_repos = list(json_data.keys())
python_commit = list(json_commits.keys())
list(set(python_repos) - set(python_commit))

[]

## Community Health

In [35]:
for file in os.listdir(path_data):
    if "community" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        health_list = []
        for repo, data in json_data.items():
            health = data.get("health_percentage")
            if isinstance(health, int):
                health_list.append(health)
            else:
                print(data)
        print(len(health_list))

cpp_community_health.json
1000
1000
csv_community_health.json
86
86
JavaScript_community_health.json
1000
1000
java_community_health.json
1000
1000
php_community_health.json
1000
1000
python_community_health.json
1000
1000


## Contributors

In [36]:
for file in os.listdir(path_data):
    if "contributors" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        cont_num_list = []
        for repo, data in json_data.items():
            elem_nr = len(data)
            if elem_nr > 0:
                cont_num_list.append(elem_nr)
            else:
                print(f"No contributors for repo: {repo}")
        print(len(cont_num_list))

cpp_contributors.json
1000
No contributors for repo: 181738376
999
csv_contributors.json
86
86
JavaScript_contributors.json
1000
No contributors for repo: 251262566
999
java_contributors.json
1000
No contributors for repo: 7813799
999
php_contributors.json
1000
1000
python_contributors.json
1000
No contributors for repo: 619885280
999


In [37]:
for file in os.listdir(path_data):
    if "organization_users" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        cont_num_list = []
        for repo, data in json_data.items():
            elem_nr = len(data)
            if elem_nr > 0:
                cont_num_list.append(elem_nr)
            else:
                print(f"No contributors for repo: {repo}")
        print(len(cont_num_list))

csv_organization_users.json
86
86
php_organization_users.json
1000
1000


In [38]:
for file in os.listdir(path_data):
    if "contributor_count" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        cont_num = 0
        for repo, data in json_data.items():
            if isinstance(data, int):
                cont_num += 1
            else:
                print(f"No contributors for repo: {repo}")
        print(cont_num)

csv_contributor_count.json
86
86
php_contributor_count.json
1000
1000


## Forks

In [39]:
for file in os.listdir(path_data):
    if "forks" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)

        print(file)
        print(len(json_data))
        fork_num_list = []
        updated_range = []
        for repo, data in json_data.items():
            elem_nr = len(data)

            if elem_nr > 0:
                fork_num_list.append(elem_nr)
            for fork in data:
                created_at = fork.get("created_at")
                create_date = datetime.strptime(created_at,'%Y-%m-%dT%H:%M:%SZ').date()
                updated_range.append(create_date)

        min_date = min(updated_range)
        print(min_date)
        max_date = max(updated_range)
        print(max_date)
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "forks",
                   "time_parameter": "created_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-12 months"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])


cpp_forks.json
1000
2022-08-27
2023-08-31
csv_forks.json
86
2022-08-27
2023-08-29
JavaScript_forks.json
1000
2017-04-07
2023-08-23
java_forks.json
1000
2019-10-24
2023-08-23
php_forks.json
1000
2022-08-27
2023-08-31
python_forks.json
1000
2021-05-27
2023-08-23


##  Issue

In [40]:
for file in os.listdir(path_data):
    if "issue" in file and not "comment" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        state_set = set()
        pull_request_set = set()
        closed_at_set = set()
        nr_issues_range = []
        for repo, data in json_data.items():
            nr_issues = len(data)
            nr_issues_range.append(nr_issues)
            for issue in data:
                state = issue.get("state")
                state_set.add(state)
                updated_at = issue.get("updated_at")
                upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
                updated_range.append(upd_date)
                pull = issue.get("pull_request")
                closed_at = issue.get("closed_at")
                if pull:
                    pull_request_set.add("pull")
                else:
                    pull_request_set.add("no pull")
                if closed_at:
                    closed_at_set.add("closed")
                else:
                    closed_at_set.add("not closed")
        print(min(nr_issues_range))
        print(max(nr_issues_range))
        print(state_set)
        print(pull_request_set)
        print(closed_at_set)
        min_date = min(updated_range)
        max_date = max(updated_range)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "issues",
                   "time_parameter": "updated_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-12 months"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])


cpp_issue.json
1000
0
480
{'closed', 'open'}
{'pull', 'no pull'}
{'not closed', 'closed'}
2023-08-26
2023-08-29
-------------------
csv_issue.json
86
0
21783
{'closed', 'open'}
{'pull', 'no pull'}
{'not closed', 'closed'}
2023-02-26
2023-08-31
-------------------
php_issue.json
1000
0
277
{'open', 'closed'}
{'pull', 'no pull'}
{'not closed', 'closed'}
2023-08-26
2023-08-28
-------------------


## Pulls

In [41]:
for file in os.listdir(path_data):
    if "pull_request" in file and not "comment" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        nr_pulls_range = []
        state_set = set()
        closed_at_set = set()
        pushed_at_set = set()
        merged_at_set = set()
        nr_issues_range = []
        for repo, data in json_data.items():
            nr_pulls = len(data)
            nr_pulls_range.append(nr_pulls)
            for pull in data:
                state = pull.get("state")
                state_set.add(state)
                updated_at = pull.get("updated_at")
                upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
                updated_range.append(upd_date)

                merged_at = pull.get("merged_at")
                closed_at = pull.get("closed_at")
                if merged_at:
                    merged_at_set.add("merged")
                else:
                    merged_at_set.add("not merged")
                if closed_at:
                    closed_at_set.add("closed")
                else:
                    closed_at_set.add("not closed")
        print(min(nr_pulls_range))
        print(max(nr_pulls_range))
        print(state_set)
        print(closed_at_set)
        print(merged_at_set)
        min_date = min(updated_range)
        max_date = max(updated_range)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "pull_request",
                   "time_parameter": "updated_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-12 months"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])

cpp_pull_requests.json
1000
0
10
{'open', 'closed'}
{'not closed', 'closed'}
{'not merged', 'merged'}
2023-08-26
2023-08-29
-------------------
csv_pull_requests.json
86
0
136
{'open', 'closed'}
{'not closed', 'closed'}
{'not merged', 'merged'}
2022-08-30
2023-08-31
-------------------
php_pull_requests.json
1000
0
8
{'open', 'closed'}
{'not closed', 'closed'}
{'not merged', 'merged'}
2023-08-26
2023-08-28
-------------------


## Release

In [42]:
path_release = os.path.join(parent_dir,"outputs", "data", "csv_release.json")

for file in os.listdir(path_data):
    if "release" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        published_range = []
        release_num_list = []
        for repo, data in json_data.items():
            release_num = len(data)
            release_num_list.append(release_num)
            for release in data:
                try:
                    published_at = release.get("published_at")
                    published_date = datetime.strptime(published_at,'%Y-%m-%dT%H:%M:%SZ').date()
                    published_range.append(published_date)
                except:
                    print(f"Att_err at repo {repo} and release  {release}")
                    
        print(min(release_num_list))
        print(max(release_num_list))
        min_date = min(published_range)
        max_date = max(published_range)
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "pull_request",
                   "time_parameter": "published_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": None}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])
            

cpp_release.json
1000
0
6898
csv_release.json
86
0
377
JavaScript_release.json
1000
Att_err at repo 39315600 and release  id
Att_err at repo 39315600 and release  tag_name
Att_err at repo 39315600 and release  prerelease
Att_err at repo 39315600 and release  published_at
0
5426
java_release.json
1000
0
6521
php_release.json
1000
0
1744
python_release.json
1000
0
1727


## Single Commits

In [43]:
for file in os.listdir(path_data):
    if "single_commits" in file  and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        language_set = set()
        archived_set = set()
        for repo, data in json_data.items():
            nr_commits = len(data)
            for commit, elem in data.items():
                if len(elem) > 1:
                    print("more than 1 elem")
                for el in elem:
                    commit_elem = el.get("commit")
                    updated_at = commit_elem.get("committer").get("date")
                    upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
                    updated_range.append(upd_date)

        min_date = min(updated_range)
        max_date = max(updated_range)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "single_commits",
                   "time_parameter": "date",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-1 month"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])

cpp_single_commits.json
626
2023-07-23
2023-08-24
-------------------
csv_single_commits.json
76
2023-07-25
2023-08-26
-------------------
JavaScript_single_commits.json
437
2023-07-23
2023-08-31
-------------------
php_single_commits.json
474
2023-07-23
2023-08-24
-------------------
python_single_commits.json
576
2023-07-23
2023-08-25
-------------------


In [44]:
file_path = os.path.join(path_data, "python_repository.json")
json_data = json_to_dict(file_path)
file_commits = os.path.join(path_data, "python_single_commits.json")
json_commits = json_to_dict(file_commits)
python_repos = list(json_data.keys())
python_commit = list(json_commits.keys())
list(set(python_repos) - set(python_commit))

['618511002',
 '53912913',
 '2154595',
 '319029846',
 '619857194',
 '45355064',
 '62743585',
 '132300505',
 '9149146',
 '387326890',
 '164564630',
 '167349656',
 '68013171',
 '42191943',
 '266613704',
 '105699750',
 '6182775',
 '164471183',
 '10464311',
 '11075275',
 '107595270',
 '115559503',
 '261515636',
 '43759462',
 '20206590',
 '66913302',
 '34757182',
 '535509546',
 '116797311',
 '60626727',
 '575048321',
 '106061907',
 '17066884',
 '122896249',
 '291703561',
 '317380990',
 '122582517',
 '21800813',
 '52581991',
 '94554082',
 '341714203',
 '122775780',
 '618029110',
 '182210458',
 '6750871',
 '2286594',
 '261247446',
 '19114225',
 '69290591',
 '136780445',
 '590604203',
 '192689297',
 '534330590',
 '2114213',
 '170067246',
 '47453302',
 '31912224',
 '24528468',
 '3959394',
 '221723816',
 '150058902',
 '51575541',
 '394541647',
 '57222302',
 '42625970',
 '149430917',
 '87520003',
 '10934235',
 '6523023',
 '3678981',
 '382756807',
 '692528',
 '595893961',
 '12484808',
 '3791384',


### Branches

In [45]:
for file in os.listdir(path_data):
    if "branches" in file and "active" not in file and "stale" not in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        print(file)
        print(len(json_data))
        updated_range = []
        language_set = set()
        archived_set = set()
        for repo, data in json_data.items():
            nr_branches = len(data)
            for branch, elem in data.items():
                if len(elem) > 1:
                    print("more than 1 elem")
                    break
                for el in elem:
                    branch_elem = el.get("commit")
                    if branch_elem:
                        commit = branch_elem.get("commit")
                        if commit:
                            committer = commit.get("committer")
                            if committer:
                                updated_at = committer.get("date")
                                if updated_at:
                                    upd_date = datetime.strptime(updated_at,'%Y-%m-%dT%H:%M:%SZ').date()
                                    updated_range.append(upd_date)

        min_date = min(updated_range)
        max_date = max(updated_range)
        print(min_date)
        print(max_date)
        print("-------------------")
        lang = file.split()[0]
        time_diff = (max_date-min_date).days
        new_row = {"language": lang, 
                   "feature": "branches",
                   "time_parameter": "updated_at",
                   "min_date": min_date.strftime("%Y-%m-%d"),
                   "max_date": max_date.strftime("%Y-%m-%d"),
                   "time_diff_days": time_diff,
                   "target_diff_days": "-1 month"}
        df_tmp = pd.DataFrame(new_row, index=[len(check_results)])
        check_results = pd.concat([check_results, df_tmp])

cpp_branches.json
1000
1998-05-20
2023-08-30
-------------------
csv_branches.json
86
1996-11-09
2023-08-27
-------------------
php_branches.json
1000
2000-05-18
2023-08-29
-------------------


In [46]:
for file in os.listdir(path_data):
    if "branches" in file and file.endswith(".json"):
        file_path = os.path.join(path_data, file)
        json_data = json_to_dict(file_path)
        if "csv" in file:
            print(file)
            print(len(json_data))
            branch_num_list = []
            for repo, data in json_data.items():
                
                elem_nr = len(data)
                if elem_nr > 0:
                    branch_num_list.append(elem_nr)
                else:
                    print(repo)
                    print(f"No branches for repo: {repo}")
            print(len(branch_num_list))
            print("-----------------")


csv_active_branches.json
86
312430210
No branches for repo: 312430210
116061796
No branches for repo: 116061796
4598769
No branches for repo: 4598769
2325298
No branches for repo: 2325298
20107959
No branches for repo: 20107959
16119670
No branches for repo: 16119670
39348622
No branches for repo: 39348622
795778
No branches for repo: 795778
23458910
No branches for repo: 23458910
8870031
No branches for repo: 8870031
239093
No branches for repo: 239093
27747508
No branches for repo: 27747508
455600
No branches for repo: 455600
11730342
No branches for repo: 11730342
4359038
No branches for repo: 4359038
35780977
No branches for repo: 35780977
3219804
No branches for repo: 3219804
396389
No branches for repo: 396389
1856976
No branches for repo: 1856976
35855357
No branches for repo: 35855357
15120676
No branches for repo: 15120676
65
-----------------
csv_branches.json
86
86
-----------------
csv_stale_branches.json
86
116061796
No branches for repo: 116061796
4598769
No branches for 

In [47]:
check_results

,language,feature,time_parameter,min_date,max_date,time_diff_days
0,cpp_repository.json,repository,updated_at,2023-08-02,2023-08-23,21
1,csv_repository.json,repository,updated_at,2023-07-21,2023-08-25,35
2,JavaScript_repository.json,repository,updated_at,2023-08-09,2023-08-23,14
3,java_repository.json,repository,updated_at,2023-08-03,2023-08-23,20
4,php_repository.json,repository,updated_at,2023-06-15,2023-08-23,69
5,python_repository.json,repository,updated_at,2023-08-15,2023-08-23,8
6,cpp_commits.json,commits,updated_at,2022-08-23,2023-08-24,366
7,csv_commits.json,commits,updated_at,2022-08-25,2023-08-25,365
8,JavaScript_commits.json,commits,updated_at,2022-08-23,2023-08-24,366
9,java_commits.json,commits,updated_at,2022-08-23,2023-08-24,366
